In [1]:
import pandas as pd
import run

In [2]:
m_path = "/home/azad/Documents/other/basilica-1.0/code_storage/data/data_sigphylo.csv"
m = pd.read_csv(m_path)
ref_path = "/home/azad/Documents/thesis/packages/pybasilica/pybasilica/data/cosmic/cosmic_catalogue.csv"
cosmic = pd.read_csv(ref_path, index_col=0)

In [3]:
one = run.fit(
    x=m, 
    k_list=[0], 
    lr=0.05, 
    n_steps=500, 
    enumer=False, 
    cluster=None, 
    groups=None, 
    beta_fixed=cosmic, 
    compile_model = False, 
    CUDA = False, 
    enforce_sparsity = False, 
    regularizer = "cosine", 
    reg_weight = 1, 
    reg_bic = False, 
    store_parameters=False, 
    verbose=False, 
    stage = "one"
    )

In [4]:
x2 = one.epsilon.astype(int)

In [5]:
two = run.fit(
    x=x2, 
    k_list=[1,2,3,4,5], 
    lr=0.05, 
    n_steps=500, 
    enumer=False, 
    cluster=None, 
    groups=None, 
    beta_fixed=None, 
    compile_model = False, 
    CUDA = False, 
    enforce_sparsity = False, 
    regularizer = "cosine", 
    reg_weight = 1, 
    reg_bic = False, 
    store_parameters=False, 
    verbose=False, 
    stage = "two"
    )

In [6]:
two.alpha

,D1,D2,D3
0,0.981127,0.014785,0.004088
1,0.908720,0.066589,0.024691
2,0.000036,0.998889,0.001075
3,0.000850,0.001177,0.997973
4,0.854010,0.145301,0.000689


In [36]:
import torch
import numpy as np

m1 = torch.tensor(m.values).float()
alpha1 = torch.tensor(one.alpha.values).float()
a1 = torch.matmul(torch.diag(torch.sum(m1, axis=1)), alpha1)

m2 = torch.tensor(x2.values).float()
alpha2 = torch.tensor(two.alpha.values).float()
a2 = torch.matmul(torch.diag(torch.sum(m2, axis=1)), alpha2)

b1 = torch.sum(a1, 1).unsqueeze(-1)
b2 = torch.sum(a2, 1).unsqueeze(-1)
b3 = b1 + b2

alpha1 = a1 / b3
alpha2 = a2 / b3

In [38]:
sample_names = list(one.alpha.index)
fixed_names = list(one.alpha.columns)
denovo_names = list(two.alpha.columns)
alpha1 = pd.DataFrame(np.array(alpha1), index=sample_names, columns=fixed_names)
alpha2 = pd.DataFrame(np.array(alpha2), index=sample_names, columns=denovo_names)
alpha = pd.concat([alpha1, alpha2], axis=1)

In [41]:
alpha

,SBS1,SBS2,SBS3,SBS4,SBS5,SBS6,SBS7a,SBS7b,SBS7c,SBS7d,...,SBS84,SBS85,SBS86,SBS87,SBS88,SBS89,SBS90,D1,D2,D3
0,0.217639,0.000013,0.000946,0.000636,0.044545,0.038298,0.000741,0.001843,0.014261,0.008778,...,0.059648,0.020507,0.003422,0.011656,0.017562,0.020152,0.000056,0.083351,0.001256,0.000347
1,0.231698,0.020178,0.000101,0.000153,0.000033,0.002415,0.039455,0.000193,0.034382,0.010747,...,0.013521,0.000225,0.000366,0.000854,0.000040,0.001904,0.018147,0.059393,0.004352,0.001614
2,0.000336,0.001148,0.000001,0.001195,0.000262,0.000109,0.000009,0.000013,0.000347,0.000272,...,0.000140,0.000517,0.000435,0.000063,0.000580,0.000348,0.000642,0.000009,0.263163,0.000283
3,0.000382,0.000433,0.000228,0.000563,0.000031,0.000301,0.000333,0.000194,0.000307,0.000153,...,0.000232,0.000220,0.000228,0.000214,0.000218,0.000434,0.000402,0.000159,0.000220,0.186898
4,0.108008,0.005588,0.000679,0.000540,0.000405,0.003287,0.031074,0.000541,0.053113,0.006119,...,0.002039,0.039502,0.000208,0.031412,0.001270,0.000058,0.001703,0.066041,0.011236,0.000053


In [42]:
torch.sum(torch.tensor(alpha.values).float(), 1).unsqueeze(-1)

tensor([[1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000]])